In [1]:
import matplotlib.pyplot as plt
import astropy
import os
import pandas as pd
import numpy as np
import xlsxwriter
import matplotlib as mpl
from astropy import io
from astropy.io import fits
from astropy import table
from astropy import wcs

In [2]:
catalogue_file = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/Catalogue/tractor-0260m062.fits'

In [3]:
image_file_g = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/Images/g-legacysurvey-0260m062-image.fits.fz'
image_file_r = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/Images/r-legacysurvey-0260m062-image.fits.fz'
image_file_z = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/Images/z-legacysurvey-0260m062-image.fits.fz'

In [4]:
catalogue = astropy.table.Table(astropy.io.fits.getdata(catalogue_file,ignore_missing_end=True))

#catalogue

In [5]:
image_z = astropy.table.Table(astropy.io.fits.getdata(image_file_z,ignore_missing_end=True))

#image_z

In [6]:
dataframe = {}
for name in catalogue.colnames:
    data = catalogue[name].tolist()
    dataframe[name] = data

In [7]:
old_catalogue = pd.DataFrame(dataframe)

#old_catalogue

hdul = astropy.io.fits.open(image_file_g)
hdul[0].data
plt.figure(figsize = (12,12))
plt.imshow(hdul[0].data, cmap='hot')

hdul = astropy.io.fits.open(image_file_r)
hdul[0].data
plt.figure(figsize = (12,12))
plt.imshow(hdul[0].data, cmap='hot')

hdul = astropy.io.fits.open(image_file_z)
hdul[0].data
plt.figure(figsize = (12,12))
plt.imshow(hdul[0].data, cmap='hot')

In [8]:
original_image = image_file_z.split(os.path.sep)[-1]

#original_image

In [9]:
hdul = astropy.io.fits.open(image_file_z)

#hdul

In [10]:
w = astropy.wcs.WCS(hdul[0].header, naxis=2)

#w

In [11]:
x, y = w.wcs_world2pix(old_catalogue['ra'], old_catalogue['dec'], 1)

In [12]:
#x

In [13]:
#y

In [14]:
new_catalogue = pd.DataFrame()
new_catalogue['objid'] = catalogue['objid']
new_catalogue['original_image'] = [original_image] * len(new_catalogue)
new_catalogue['flux_g'] = catalogue['flux_g']
new_catalogue['flux_r'] = catalogue['flux_r']
new_catalogue['flux_z'] = catalogue['flux_z']
new_catalogue['x'] = x
new_catalogue['y'] = y
new_catalogue['ra'] = catalogue['ra']
new_catalogue['dec'] = catalogue['dec']

#new_catalogue

In [15]:
def convert_tractor_catalogue(catalogue_file, image_file, image_name=''):
    """
    Converts a tractor fits file to a pandas dataframe to be given
    directly to an ImageDataset object.

    Parameters
    ----------
    catalogue_files : string
        tractor catalogue in fits table format 
    image_file:
        The image corresponding to this catalogue (to extract pixel information
        and naming information)
    """

    catalogue = astropy.table.Table(astropy.io.fits.getdata(catalogue_file,ignore_missing_end=True))

    dataframe = {}
    for name in catalogue.colnames:
        data = catalogue[name].tolist()
        dataframe[name] = data
    
    old_catalogue = pd.DataFrame(dataframe)
    hdul = astropy.io.fits.open(image_file)

    if len(image_name) == 0:
        original_image = image_file.split(os.path.sep)[-1]
    else:
        original_image = image_name
    
    #w = astropy.wcs.WCS(hdul[0].header, naxis=2)
    #x, y = w.wcs_world2pix(old_catalogue['ra'], old_catalogue['dec'], 1)
    
    new_catalogue = pd.DataFrame()
    new_catalogue['objid'] = old_catalogue['objid']
    new_catalogue['original_image'] = [original_image] * len(new_catalogue)
    new_catalogue['flux_g'] = old_catalogue['flux_g']
    new_catalogue['flux_r'] = old_catalogue['flux_r']
    new_catalogue['flux_z'] = old_catalogue['flux_z']
    new_catalogue['x'] = old_catalogue['bx'].astype('int')
    new_catalogue['y'] = old_catalogue['by'].astype('int')
    new_catalogue['ra'] = old_catalogue['ra']
    new_catalogue['dec'] = old_catalogue['dec']
    
    new_catalogue['type'] = old_catalogue['type']
    
    return new_catalogue

In [16]:
image_name = 'legacysurvey-0260m062-image.fits.fz'

In [17]:
prod = convert_tractor_catalogue(catalogue_file, image_file_z, image_name)

In [18]:
catalogue_no_PSFs = prod[prod.type.astype("S") != b'PSF ']

#catalogue_no_PSFs

In [22]:
catalogue_no_PSFs.sort_values('flux_r', ascending=False)

,objid,original_image,flux_g,flux_r,flux_z,x,y,ra,dec,type
594,594,legacysurvey-0260m062-image.fits.fz,317.535095,720.682556,1307.191528,337,369,26.126581,-6.354089,DEV
6095,6095,legacysurvey-0260m062-image.fits.fz,168.212524,284.601593,412.424927,994,2486,26.078486,-6.199997,EXP
2709,2709,legacysurvey-0260m062-image.fits.fz,51.362320,146.547363,276.822815,3216,1209,25.915781,-6.292911,DEV
5667,5667,legacysurvey-0260m062-image.fits.fz,71.172943,143.411240,247.036163,1088,2359,26.071583,-6.209265,EXP
2471,2471,legacysurvey-0260m062-image.fits.fz,43.115738,121.118385,238.656158,2859,1137,25.941948,-6.298205,DEV
...,...,...,...,...,...,...,...,...,...,...
1387,1387,legacysurvey-0260m062-image.fits.fz,-0.002538,0.027460,0.509285,3125,619,25.922422,-6.335852,REX
4825,4825,legacysurvey-0260m062-image.fits.fz,0.005111,0.024595,0.499707,1270,1938,26.058274,-6.239884,REX
8559,8559,legacysurvey-0260m062-image.fits.fz,0.000000,0.000000,0.000000,337,369,26.126580,-6.354087,DUP
2941,2941,legacysurvey-0260m062-image.fits.fz,0.109954,-0.004085,0.056334,1629,1271,26.032034,-6.288446,REX


In [23]:
test_catalogue_0260m062_500 = catalogue_no_PSFs.sort_values('flux_r', ascending=False)[0:500]

#TOP 500!
test_catalogue_0260m062_500

,objid,original_image,flux_g,flux_r,flux_z,x,y,ra,dec,type
594,594,legacysurvey-0260m062-image.fits.fz,317.535095,720.682556,1307.191528,337,369,26.126581,-6.354089,DEV
6095,6095,legacysurvey-0260m062-image.fits.fz,168.212524,284.601593,412.424927,994,2486,26.078486,-6.199997,EXP
2709,2709,legacysurvey-0260m062-image.fits.fz,51.362320,146.547363,276.822815,3216,1209,25.915781,-6.292911,DEV
5667,5667,legacysurvey-0260m062-image.fits.fz,71.172943,143.411240,247.036163,1088,2359,26.071583,-6.209265,EXP
2471,2471,legacysurvey-0260m062-image.fits.fz,43.115738,121.118385,238.656158,2859,1137,25.941948,-6.298205,DEV
...,...,...,...,...,...,...,...,...,...,...
8291,8291,legacysurvey-0260m062-image.fits.fz,0.891018,2.086115,4.284080,675,3403,26.101856,-6.133251,REX
4974,4974,legacysurvey-0260m062-image.fits.fz,0.721040,2.083005,5.874446,517,2012,26.113394,-6.234514,EXP
5297,5297,legacysurvey-0260m062-image.fits.fz,0.866198,2.076526,6.970213,491,2141,26.115329,-6.225094,DEV
6946,6946,legacysurvey-0260m062-image.fits.fz,1.133894,2.072207,4.155129,1970,2812,26.007054,-6.176279,EXP


In [24]:
test_catalogue_0260m062_500.to_csv ('/home/verlon/Desktop/Astronomaly/Data/test_catalogue_0260m062_500.csv', index = True, header=True)